In [1]:
# ## Installing TA-Lib
# url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
# !curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
# url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
# !curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
# import talib

# Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import talib
import os
from tqdm.keras import TqdmCallback

# from keras_tqdm import TQDMNotebookCallback

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, root_mean_squared_error

C:\Users\bpadmin\AppData\Local\Temp\ipykernel_19444\906520915.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\bpadmin\anaconda3\envs\tf_portfolio\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
  details = tf.config.experimental.get_device_details(gpu_devices[0])
  details.get('device_name', 'Unknown GPU')

print (details)

{'device_name': 'NVIDIA GeForce RTX 4060 Laptop GPU', 'compute_capability': (8, 9)}


# functions/utils

## WindoeGenerator

In [4]:
# data_returns.mean()
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result


# map() applies a provided function to every datapoint in the dataset

## add_rsi_indicator()

In [5]:
def add_rsi_indicator(symbol_df):
    df = symbol_df.copy()
    df['rsi_val'] = talib.RSI(df.close)
    df = df.dropna(axis = 0, how = 'any')
    return df

## get_returns()

In [6]:
def get_returns(df):
  data1 = df['close'][1:].to_numpy()
  data2 = df['close'][:-1].to_numpy()
  df = df.reset_index(drop = True)
  df.loc[1:, 'returns'] = data1/data2
  df = df.dropna(axis = 0, how = 'any').reset_index(drop = True)
  return df

## get_log_returns()

In [7]:
def get_log_returns(df):
  data1 = df['close'][1:].to_numpy()
  data2 = df['close'][:-1].to_numpy()
  df = df.reset_index(drop = True)
  df.loc[1:, 'returns'] = data1/data2
  df['log_returns'] = np.log(df['returns'])
  df = df.dropna(axis = 0, how = 'any').reset_index(drop = True)
  return df

## get_log_returns_v2()

In [8]:
def get_log_returns_v2(df, horizon):
  data1 = df['close'][horizon:].to_numpy()
  data2 = df['close'][:-horizon].to_numpy()
  df = df.reset_index(drop = True)
  df.loc[horizon:, 'returns'] = data1/data2
  df.loc[horizon*2:, 'expected_returns'] = df.loc[:,'returns'].shift(-horizon)
  df['log_returns'] = np.log(df['returns'])
  df['log_expected_returns'] = np.log(df['expected_returns'])
  df = df.dropna(axis = 0, how = 'any').reset_index(drop = True)
  return df

## train_val_test_split()

In [9]:
def create_seqs(df, feat_cols,feat_len = 10, horizon = 5):
  df_np = df.loc[:,feat_cols].to_numpy()
  test_x, test_y = [], []

  total_window = feat_len + horizon
  for step in range(len(df_np)- total_window):

    # test_x.append(df_np[step: step+feat_len])
    test_x.append(np.expand_dims(df_np[step: step+feat_len], axis = 0))
    # test_y.append([df_np[step+total_window,[1]]])
    test_y.append(np.expand_dims(df_np[step+total_window-1,[1]], axis=1))

  return (test_x, test_y)

def train_val_test_split(symbol_dfs, feat_cols, split_sizes = [0.7, 0.1, 0.2],feat_len = 10, horizon = 5):
  size = symbol_dfs.get(next(iter(symbol_dfs))).get('full').shape[0]
  for symbol in list(symbol_dfs.keys()):
    symbol_df = symbol_dfs.get(symbol).get('full')

    #train data
    train_df = symbol_df.iloc[:int(size*0.7)].reset_index(drop = True)
    train_sqs = create_seqs(train_df, feat_cols,feat_len, horizon)
    train_ds = tf.data.Dataset.from_tensor_slices(train_sqs)
    symbol_dfs[symbol]['train'] = train_ds

    #validation data
    val_df = symbol_df.iloc[int(size*0.7):int(size*0.8)].reset_index(drop = True)
    val_sqs = create_seqs(val_df, feat_cols,feat_len, horizon)
    val_ds = tf.data.Dataset.from_tensor_slices(val_sqs)
    symbol_dfs[symbol]['val'] = val_ds

    #test data
    test_df = symbol_df.iloc[int(size*0.8):].reset_index(drop = True)
    test_sqs = create_seqs(test_df, feat_cols,feat_len, horizon)
    test_ds = tf.data.Dataset.from_tensor_slices(test_sqs)
    symbol_dfs[symbol]['test'] = test_ds

  return symbol_dfs

In [10]:
# def create_seqs(df, feat_cols,feat_len = 10, shift = 4, label_len = 1):
#   df_np = df.loc[:,feat_cols].to_numpy()
#   test_x, test_y = [], []

#   total_window = feat_len + shift + label_len
#   for step in range(len(df_np)- total_window):

#     # test_x.append(df_np[step: step+feat_len])
#     test_x.append(np.expand_dims(df_np[step: step+feat_len], axis = 0))
#     # test_y.append([df_np[step+total_window,[1]]])
#     test_y.append(np.expand_dims(df_np[step+total_window,[1]], axis=1))

#   return (test_x, test_y)

# def train_val_test_split(symbol_dfs, feat_cols, split_sizes = [0.7, 0.1, 0.2],feat_len = 10, shift = 4, label_len = 1):
#   size = symbol_dfs.get(next(iter(symbol_dfs))).get('full').shape[0]
#   for symbol in list(symbol_dfs.keys()):
#     symbol_df = symbol_dfs.get(symbol).get('full')

#     #train data
#     train_df = symbol_df.iloc[:int(size*0.7)].reset_index(drop = True)
#     train_sqs = create_seqs(train_df, feat_cols,feat_len, shift, label_len)
#     train_ds = tf.data.Dataset.from_tensor_slices(train_sqs)
#     symbol_dfs[symbol]['train'] = train_ds

#     #validation data
#     val_df = symbol_df.iloc[int(size*0.7):int(size*0.8)].reset_index(drop = True)
#     val_sqs = create_seqs(val_df, feat_cols,feat_len, shift, label_len)
#     val_ds = tf.data.Dataset.from_tensor_slices(val_sqs)
#     symbol_dfs[symbol]['val'] = val_ds

#     #test data
#     test_df = symbol_df.iloc[int(size*0.8):].reset_index(drop = True)
#     test_sqs = create_seqs(test_df, feat_cols,feat_len, shift, label_len)
#     test_ds = tf.data.Dataset.from_tensor_slices(test_sqs)
#     symbol_dfs[symbol]['test'] = test_ds

#   return symbol_dfs

## model_compile_and_fit()

In [11]:
def create_lstm(model_name, input_shape):
  tf.keras.backend.clear_session()
  lstm_model = tf.keras.models.Sequential([

    tf.keras.layers.Input(shape=input_shape, name='Seq_Input'),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(8),

    tf.keras.layers.Dense(units=1)
  ], name = model_name)

  return lstm_model

def model_compile_and_fit(model_name, train_data, val_data, max_epochs = 20, patience=2):

  X, _ = next(iter(train_data))
  input_shape = (X.shape[1], X.shape[2])
  model = create_lstm(model_name,input_shape)

  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  # tqdm_params = {'metrics': [tf.keras.metrics.MeanAbsoluteError()]}

  history = model.fit(train_data, epochs=max_epochs,
                      validation_data=val_data,
                      verbose = 0,
                      callbacks=[early_stopping, TqdmCallback(verbose=2)])
  return history, model

## evaluate_symbol()

In [12]:
def tfds_to_numpy_label(ds):
  test_ds_label_list = []
  for X, y in (ds):
    test_ds_label_list.append(y.numpy().flatten()[0])

  ds_true = np.array(test_ds_label_list)
  return ds_true

def evaluate_symbol(symbol_dfs):
  for symbol in list(symbol_dfs.keys()):
    print('Symbol : {}'.format(symbol))
    train_ds = symbol_dfs.get(symbol).get('train')
    val_ds = symbol_dfs.get(symbol).get('val')
    test_ds = symbol_dfs.get(symbol).get('test')

    hist, model = model_compile_and_fit('test_model', train_ds, val_ds, max_epochs = 5, patience=2)

    symbol_dfs[symbol]['model'] = model
    symbol_dfs[symbol]['history'] = hist

    preds = model.predict(test_ds)
    test_true = tfds_to_numpy_label(test_ds)

    mse = mean_squared_error(test_true, preds)
    symbol_dfs[symbol]['evaluation']['mse'] = mse
    rmse = root_mean_squared_error(test_true, preds)
    symbol_dfs[symbol]['evaluation']['rmse'] = rmse

    print('mse : {} | rmse : {}'.format(mse, rmse))
    print('')
  return symbol_dfs


## save_results

In [13]:
def save_results(symbol_dfs, path):

  if os.path.exists(path):
    res_df = pd.read_excel(path ,index_col=None)
  else:
    res_df = pd.DataFrame()
  for symbol in list(symbol_dfs.keys()):
    res = {}
    res['symbol'] = symbol
    # res['model_history'] = symbol_dfs.get(symbol).get('history')
    res['mse'] = symbol_dfs.get(symbol).get('evaluation').get('mse')
    res['rmse'] = symbol_dfs.get(symbol).get('evaluation').get('rmse')

    res_df = pd.concat([res_df,pd.DataFrame([res])], ignore_index = True)

  res_df.to_csv(path, index = False)
  print('saved results @ {}'.format(path))

# initialize

In [14]:
data_path = '../data/processed/close_price_data_50.csv'
data = pd.read_csv(data_path, index_col= False)
data = data.rename({'index':'timestamp'},axis =1 )
data = data.dropna(axis = 1, how = 'any')
data.head(5)

,timestamp,TSLA,META,MSFT,AAPL,NVDA,GOOGL,AMZN,AMD,INTC,...,JPM,C,LUV,GM,KDP,NCLH,QCOM,TXN,PLUG,SBUX
0,2014-02-07 00:00:00-05:00,12.435333,64.320000,30.785587,16.368795,3.743081,29.465466,18.054001,3.47,18.418928,...,42.844234,39.150589,19.640068,27.489239,6.269822,35.209999,55.553337,32.300919,3.10,30.928185
1,2014-02-10 00:00:00-05:00,13.104000,63.549999,30.987686,16.662035,3.754874,29.352604,18.043501,3.63,18.479792,...,42.935032,39.134720,19.264313,26.568119,6.245455,35.619999,55.762371,32.616619,3.55,31.245647
2,2014-02-11 00:00:00-05:00,13.108000,64.849998,31.299236,16.881582,3.832708,29.784285,18.089500,3.70,18.616737,...,43.457157,39.404495,19.539259,26.834572,6.271106,36.250000,56.456692,33.209515,3.51,31.120329
3,2014-02-12 00:00:00-05:00,13.021333,64.449997,31.551868,16.880314,3.969505,29.696947,17.462500,3.69,18.677601,...,43.525261,39.642551,19.438437,27.070557,6.410890,35.889999,57.053936,33.340393,3.74,30.873867
4,2014-02-13 00:00:00-05:00,13.308667,67.330002,31.669739,17.148363,4.094512,30.027529,17.860001,3.70,18.791718,...,43.911179,39.563210,19.429276,26.796497,6.421149,34.700001,57.068897,33.679199,3.94,31.199690


In [15]:
# # taking few columns to test vscode setup

# data = data.iloc[:,:5]
# data

In [16]:
symbols = data.columns[1:]
symbols

Index(['TSLA', 'META', 'MSFT', 'AAPL', 'NVDA', 'GOOGL', 'AMZN', 'AMD', 'INTC',
       'F', 'XOM', 'MU', 'T', 'DIS', 'BA', 'JNJ', 'VZ', 'ENPH', 'PFE', 'CMCSA',
       'NFLX', 'BAC', 'RTX', 'KO', 'CCL', 'BMY', 'CSCO', 'ON', 'JPM', 'C',
       'LUV', 'GM', 'KDP', 'NCLH', 'QCOM', 'TXN', 'PLUG', 'SBUX'],
      dtype='object')

In [17]:
# this dict stores all the dataframes belonging to every symbol - every symbol is processed seperatly
symbol_dfs = {symbol :
 {
    'full': '',
    'train': '',
    'val':'' ,
    'test' : '',
    'model':'',
    'history':'',
    'evaluation':
   {
       'mse':'',
       'rmse':''
       }
  }
  for symbol in symbols}

# Preprocessing

## only price

In [18]:
symbol_close_dfs = {}
for symbol in data.columns[1:]:
  symbol_df = data[symbol].to_frame().rename({symbol:'close'}, axis = 1)
  # symbol_df = add_rsi_indicator(symbol_df)
  symbol_df = get_returns(symbol_df)

  #store dataframe in dict
  symbol_close_dfs['{}_close'.format(symbol)] = symbol_df


## with_rsi

In [19]:
symbol_close_dfs = {symbol : {} for symbol in symbols}
for symbol in data.columns[1:]:
  symbol_df = data[symbol].to_frame().rename({symbol:'close'}, axis = 1)
  symbol_df = add_rsi_indicator(symbol_df)
  symbol_df = get_returns(symbol_df)

  #store dataframe in dict
  symbol_close_dfs[symbol]['full'] = symbol_df


In [20]:
def preprocess_data(data, symbol_dfs):
  for symbol in symbols:
    symbol_df = data[symbol].to_frame().rename({symbol:'close'}, axis = 1)
    symbol_df = add_rsi_indicator(symbol_df)
    symbol_df = get_log_returns_v2(symbol_df, horizon = 5)

    #store dataframe in dict
    symbol_dfs[symbol]['full'] = symbol_df

  return symbol_dfs

In [21]:
symbols_dfs = preprocess_data(data, symbol_dfs)

# train test split

In [22]:
feat_cols = ['rsi_val', 'log_returns']

symbol_dfs = train_val_test_split(symbol_dfs, feat_cols, split_sizes = [0.7, 0.1, 0.2], feat_len = 10, horizon = 5)

# Evaluation

In [23]:
symbol_dfs = evaluate_symbol(symbol_dfs)

Symbol : TSLA


100%|██████████| 1.71k/1.71k [00:09<00:00, 185batch/s, loss=0.00797, mean_absolute_error=0.0646]oss=0.0495, val_mean_absolute_error=0.194]
100%|██████████| 1.71k/1.71k [00:05<00:00, 286batch/s, loss=0.00733, mean_absolute_error=0.0611]oss=0.0216, val_mean_absolute_error=0.126]
100%|██████████| 1.71k/1.71k [00:05<00:00, 303batch/s, loss=0.00693, mean_absolute_error=0.059]oss=0.0163, val_mean_absolute_error=0.107] 
100%|██████████| 1.71k/1.71k [00:06<00:00, 275batch/s, loss=0.00696, mean_absolute_error=0.0593]oss=0.0182, val_mean_absolute_error=0.112]
100%|██████████| 1.71k/1.71k [00:06<00:00, 269batch/s, loss=0.00667, mean_absolute_error=0.0576]oss=0.0104, val_mean_absolute_error=0.0837]
100%|██████████| 5/5 [00:33<00:00,  6.70s/epoch, loss=0.00667, mean_absolute_error=0.0576, val_loss=0.0104, val_mean_absolute_error=0.0837]


480/480 [==============================] - 1s 1ms/step
mse : 0.01763206789533317 | rmse : 0.1327857970391908

Symbol : META


100%|██████████| 1.71k/1.71k [00:07<00:00, 222batch/s, loss=0.00293, mean_absolute_error=0.0405]oss=0.0142, val_mean_absolute_error=0.11]
100%|██████████| 1.71k/1.71k [00:06<00:00, 284batch/s, loss=0.00248, mean_absolute_error=0.0371]oss=0.0075, val_mean_absolute_error=0.0777]
100%|██████████| 1.71k/1.71k [00:05<00:00, 314batch/s, loss=0.00235, mean_absolute_error=0.0361]oss=0.00606, val_mean_absolute_error=0.069]
100%|██████████| 1.71k/1.71k [00:05<00:00, 313batch/s, loss=0.00232, mean_absolute_error=0.0358]oss=0.00523, val_mean_absolute_error=0.0635]
100%|██████████| 1.71k/1.71k [00:05<00:00, 304batch/s, loss=0.0023, mean_absolute_error=0.0356] ss=0.00473, val_mean_absolute_error=0.06]   
100%|██████████| 5/5 [00:30<00:00,  6.07s/epoch, loss=0.0023, mean_absolute_error=0.0356, val_loss=0.00473, val_mean_absolute_error=0.06]


480/480 [==============================] - 1s 1ms/step
mse : 0.010284031818106231 | rmse : 0.10141021555102933

Symbol : MSFT


...

480/480 [==============================] - 1s 1ms/step
mse : 0.0015403442539982698 | rmse : 0.03924721969768393

Symbol : PLUG


100%|██████████| 1.71k/1.71k [00:06<00:00, 257batch/s, loss=0.0145, mean_absolute_error=0.0877]oss=0.218, val_mean_absolute_error=0.443]
100%|██████████| 1.71k/1.71k [00:05<00:00, 305batch/s, loss=0.0132, mean_absolute_error=0.0829]oss=0.0878, val_mean_absolute_error=0.272]
100%|██████████| 1.71k/1.71k [00:05<00:00, 305batch/s, loss=0.0126, mean_absolute_error=0.0797]oss=0.0672, val_mean_absolute_error=0.233]
100%|██████████| 1.71k/1.71k [00:05<00:00, 315batch/s, loss=0.0124, mean_absolute_error=0.0786]oss=0.0466, val_mean_absolute_error=0.189]
100%|██████████| 1.71k/1.71k [00:05<00:00, 311batch/s, loss=0.0123, mean_absolute_error=0.078] ss=0.0389, val_mean_absolute_error=0.17]  
100%|██████████| 5/5 [00:28<00:00,  5.77s/epoch, loss=0.0123, mean_absolute_error=0.078, val_loss=0.0389, val_mean_absolute_error=0.17]


480/480 [==============================] - 1s 1ms/step
mse : 0.04264336439014573 | rmse : 0.2065026982635959

Symbol : SBUX


100%|██████████| 1.71k/1.71k [00:06<00:00, 247batch/s, loss=0.00174, mean_absolute_error=0.0294]oss=0.00194, val_mean_absolute_error=0.0359]
100%|██████████| 1.71k/1.71k [00:05<00:00, 306batch/s, loss=0.00155, mean_absolute_error=0.0275]oss=0.00155, val_mean_absolute_error=0.0321]
100%|██████████| 1.71k/1.71k [00:05<00:00, 303batch/s, loss=0.00144, mean_absolute_error=0.0263]oss=0.00131, val_mean_absolute_error=0.0292]
100%|██████████| 1.71k/1.71k [00:05<00:00, 312batch/s, loss=0.00139, mean_absolute_error=0.0256]oss=0.00122, val_mean_absolute_error=0.0278]
100%|██████████| 1.71k/1.71k [00:05<00:00, 311batch/s, loss=0.00133, mean_absolute_error=0.0251]oss=0.00112, val_mean_absolute_error=0.0262]
100%|██████████| 5/5 [00:29<00:00,  5.84s/epoch, loss=0.00133, mean_absolute_error=0.0251, val_loss=0.00112, val_mean_absolute_error=0.0262]


480/480 [==============================] - 1s 1ms/step
mse : 0.0019884268993028606 | rmse : 0.044591780624940965



In [24]:
import datetime
dtime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
file_name = '{}_NUS_expected_returns_lstm_v2.csv'.format(dtime)
res_local_path = '../Results'

save_path = os.path.join(res_local_path, file_name)
save_results(symbols_dfs,save_path)

saved results @ ../Results\2024-01-31_13-08_NUS_expected_returns_lstm_v2.csv


# WORK HEREEEEEE

In [25]:
import sys
sys.exit()

SystemExit: 

c:\Users\bpadmin\anaconda3\envs\tf_portfolio\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
test_df = symbols_dfs.get('TSLA').get('full')
test_df.head(16)

In [ ]:
tr_ds = symbols_dfs.get('TSLA').get('train')
tr_ds

In [ ]:
tr_d

In [ ]:
X, y =next(iter(tr_ds))
X.shape
input_shape =

In [ ]:
horizon = 5
data1 = test_df['close'][horizon:].to_numpy()
data2 = test_df['close'][:-horizon].to_numpy()

a = data1/data2
a

In [ ]:
np.roll(a, shift, axis=None)

In [ ]:
horizon = 5
test_df.loc[:len(test_df)-1-horizon,'returns'] = test_df.loc[:,'returns'].shift(-horizon)
test_df

In [ ]:
test_df.idxmax(axis = 1)

In [ ]:
horizon = 5
test_df.loc[:len(test_df)-1-horizon,'close']

In [ ]:
15.398000/16.414000

In [ ]:
list(symbol_dfs.keys())

In [ ]:
def evaluate_symbol(symbol_dfs):
  for symbol in list(symbol_dfs.keys()):

    train_ds = symbol_dfs.get(symbol).get('train')
    val_ds = symbol_dfs.get(symbol).get('val')
    test_ds = symbol_dfs.get(symbol).get('test')

    hist, model = model_compile_and_fit('test_model', train_ds, val_ds, max_epochs = 5, patience=2)

    symbol_dfs[symbol]['model'] = model
    symbol_dfs[symbol]['history'] = hist

    preds = model.predict(test_ds)
    test_true = tfds_to_numpy_label(test_ds)

    mse = mean_squared_error(test_true, preds)
    symbol_dfs[symbol]['evaluation']['mse'] = mse
    rmse = mean_squared_error(test_true, preds, squared = False)
    symbol_dfs[symbol]['evaluation']['rmse'] = rmse

  return symbol_dfs


In [ ]:
symbol_dfs.get(next(iter(symbol_dfs))).get('full')

In [ ]:
train_ds = symbol_dfs.get(next(iter(symbol_dfs))).get('train')
val_ds = symbol_dfs.get(next(iter(symbol_dfs))).get('val')
test_ds = symbol_dfs.get(next(iter(symbol_dfs))).get('test')

In [ ]:
train_ds

In [ ]:
hist, model = model_compile_and_fit('test_model', train_ds, val_ds, max_epochs = 5, patience=2)

In [ ]:
hist_dataframe = pd.DataFrame(data = hist.history,columns = hist.history.keys())
hist_dataframe

In [ ]:
import plotly.express as px

fig = px.line(hist_dataframe, y="loss")
fig.show()

In [ ]:
train_ds.take(1)

In [ ]:
preds = model.predict(test_ds)
# preds

In [ ]:
model.summary()

In [ ]:
preds.shape

In [ ]:
test_ds.take(1)

In [ ]:
test_ds_label_list = []
for X, y in (test_ds):
  test_ds_label_list.append(y.numpy().flatten()[0])

In [ ]:
def tfds_to_numpy_label(ds):
  test_ds_label_list = []
  for X, y in (ds):
    test_ds_label_list.append(y.numpy().flatten()[0])

  ds_true = np.array(test_ds_label_list)
  return ds_true

In [ ]:
test_ds_true = np.array(test_ds_label_list)
test_ds_true[:5]

In [ ]:
preds[:5]

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_ds_true, preds), mean_squared_error(test_ds_true, preds, squared = False)

# ENDDDD